In [ ]:
import LinearAlgebra

In [ ]:
import Random

In [ ]:
import BenchmarkTools

In [ ]:
import StaticArrays
# details on staticarrays here https://m3g.github.io/JuliaNotes.jl/stable/immutable/

In [ ]:
import NBInclude

In [ ]:
import LinearAlgebra

In [ ]:
# import the code from the l,d motif simulation notebook
NBInclude.@nbinclude("simulate_ld_motif.ipynb")

# try calculating something like a gradient
which direction should we turn each dimension to increase the score?

In [ ]:
# it is written in such a way that it is continuously defined
function Score(Starts_inp, Seqs=sequences, Length=length(motif); ln=false)
    # MODIFY the inputs if they are out of bounds :-)
    ### TO DO: return a very large number of the starts are out of bounds
    # in the mean time, just fix out of bounds errors
    for i in 1:length(Starts_inp)
        if (Starts_inp[i] < 1)
            Starts_inp[i] = 1
        elseif (Starts_inp[i] > (length(Seqs[1]) - Length + 1))
            Starts_inp[i] = length(Seqs[1]) - Length + 1
        end
            
    end
    maxScore = Length * length(Seqs)
        
    seqsMatrix = permutedims(reduce(hcat, map((s, i) -> s[i:(i + Length - 1)], Seqs, Starts_inp)))
    
    # find the most common element in each column
    mostCommon = mapslices(StatsBase.mode, seqsMatrix, dims = 1)
    
    # now count how many sequences are equal to the consensus
    thisScore = sum(map((i, j) -> sum(i .== j), eachslice(seqsMatrix, dims = 2), mostCommon))
    
    if ln
        return(-1 * log(maxScore - thisScore + 1))
    else
    # make the minimum (best) score 1
        return(maxScore - thisScore + 1)
    end
end

In [ ]:
# calculate a partial score in the plus and minus direction
# return positive if going in the positive direction increases the score
# return negative if going in the negative direction increases the score
# the value given is the sum score difference gained from going x-1 -> x -> x+1
#### TO DO: deal with edges of the search space
# https://www2.atmos.umd.edu/~ekalnay/syllabi/AOSC614/NWP-CH03-2-2.pdf
function Grad(Starts_inp, seq, dt=1)
    backward = copy(Starts_inp)
    forward = copy(Starts_inp)
    
    backward[seq] -= dt
    forward[seq] += dt
    
    #score1 = Score(Starts_inp) - Score(backward)
    #score2 = Score(forward) - Score(Starts_inp)
    return((Score(forward; ln = true) - Score(backward; ln = true))/(2*dt))
end

In [ ]:
function CalcGrad(StartsInp, dt=1)
    return Grad.((StartsInp,), 1:length(StartsInp), (dt,))
end

In [ ]:
function Grad2(Starts_inp, seq, dt=1)
    backward = copy(Starts_inp)
    forward = copy(Starts_inp)
    
    backward[seq] -= dt
    forward[seq] += dt
    
    #score1 = Score(Starts_inp) - Score(backward)
    #score2 = Score(forward) - Score(Starts_inp)
    return(Score(forward; ln = true) - 2 * Score(Starts_inp; ln = true) + Score(backward; ln = true)/(dt^2))
end

In [ ]:
function CalcGrad2(StartsInp, dt=1)
    return Grad2.((StartsInp,), 1:length(StartsInp), (dt,))
end

# hamiltonian eqs
taken from wikipedia https://en.wikipedia.org/wiki/Hamiltonian_Monte_Carlo

In [ ]:
function H(particle, velocity)
    return(U(particle) + 1/2 * velocity' * inv(LinearAlgebra.I) * velocity)
end

In [ ]:
function U(particle)
    return(-1 * log(Score(particle)))
end

In [ ]:
function leap(particle, velocity, dt, iter)
    particle_sample = []
    # initial particle position and velocity
    xnt = copy(particle)
    pnt = copy(velocity)
    
    # iterate for the number of time steps desired
    # TODO: add stopping criterion
    for t in 1:iter
        # do the first half-step
        # update the particles momentum by looking at the previous moments
        # and adding half-ish the change in the potential energy
        pn_t_dt2 = pnt .- (dt/2) .* CalcGrad(xnt, dt)
        
        # update the particle position after t time
        # by using the new momentum, found in the previous half step
        xn_t_dt = xnt .+ dt * inv(LinearAlgebra.I) * pn_t_dt2
        # round this value to an integer, since our scorer only works
        # on discrete values
        ### TODO: add a bounds check here to prevent from going out of bounds
        xn_t_dt = Int.(round.(xn_t_dt))
        
        # update the momentum again, this time by evaluating the gradient at the new point
        pn_t_dt = pn_t_dt2 .- (dt/2) .* CalcGrad(xn_t_dt, dt)
        
        # do a metropolis-hastings step
        
        # compute the ratio of hamiltonian new/old like on the wiki page
        rat = (exp(-1 * H(xn_t_dt, pn_t_dt)))/(exp(-1 * H(xnt, pnt)))
        α = minimum([1, rat])
        
        # if our random number is less than alpha, accept the new proposal
        if rand(1)[1] .< α
            xnt = xn_t_dt
            pnt = pn_t_dt
            append!(particle_sample, (xnt,))
        else
            # otherwise, repeat the prior sample
            append!(particle_sample, (xnt,))
        end
        
        #### TODO:
        # check if we are at a boundary. if so, reverse the sign of the velocity for that dimension
        
        
        #=
        # print to double check computation
        println(" t ==== ", t)
        println(pn_t_dt2)
        println(xn_t_dt)
        println(pn_t_dt)
        println(rat)
        println()=#
    end
    return(particle_sample)
end

In [ ]:
function Roll(Position, Velocity, tmax = 10)
    # copy the inputs so we don't modify them on accident
    position = copy(Position)
    
    velocity = copy(Velocity)
    score = zeros(Int64, tmax)
    # continue rolling until t > 10 or velocity is 0
    t = 1
    while (t <= tmax) & (any(velocity .!= 0))
        # calculate the current score
        score[t] = Score(position)
        
        println(" ==== t ==== ", t)
        println("pos: ",  position)
        println("vel: ", velocity)
        println("sco: ", score[t])
        println("grd: ", CalcGrad(position))
        println()
        
        # "integrate" over the scores in the path of the particle
        # only move in discrete steps, though
        # will need to add leap-frog steps in discrete space later, though
        
        # strategy here will be to ALWAYS move in the direction of the maximum gradient,
        # and then sometimes move in the other directions, randomly
        move_prob = abs.(float.(CalcGrad(position)))
        move_prob ./= maximum(move_prob)
        move_accep = rand(length(velocity)) .<= move_prob
        # update the position using the velocity - go in the direction of the velocity
        position .+= sign.(velocity) .* move_accep
        
        # shrink the velocity by one in each dimension to simulate friction
        velocity .-= CalcGrad(position) .* move_accep
        t += 1
    end
end

In [ ]:
# set the seed
Random.seed!(100)
# use tuple unpacking to get some test values
# NumberOfSequences, LengthMotif, LengthSequences, Distance
motif, motif_starts, motifs_implanted, sequences = GenerateTestData_ld(3, 14, 100, 0)

In [ ]:
Random.seed!(100)
Roll([2,2,2], [3, 3, 3], 5)

In [ ]:
#BenchmarkTools.@btime Roll([54, 26, 6], [3, 3, 3], 20)

In [ ]:
# set the seed
Random.seed!(100)
# use tuple unpacking to get some test values
# NumberOfSequences, LengthMotif, LengthSequences, Distance
motif, motif_starts, motifs_implanted, sequences = GenerateTestData_ld(6, 14, 1000, 0)

In [ ]:
leap(repeat([500], length(sequences)), repeat([1], length(sequences)), 1, 200)

In [ ]:
BenchmarkTools.@btime leap(repeat([500], length(sequences)), repeat([1], length(sequences)), 1, 100)